# Aggregation & Grouping

Grouping and aggregating helps you **summarize your data** — like answering:  
> “What’s the average salary *per department*?”  
> “How many users joined the Gym *per month*?”

---

## `.groupby()` Function

df.groupby() is used to group rows of a DataFrame based on the values in one or more columns, which allows you to then perform aggregate functions (like sum(), mean(), count(), etc.) on each group.
Consider this DataFrame:

```python 
df = pd.DataFrame({
    "Department": ["HR", "HR", "IT", "IT", "Marketing", "Marketing", "Sales", "Sales"],
    "Team": ["A", "A", "B", "B", "C", "C", "D", "D"],
    "Gender": ["M", "F", "M", "F", "M", "F", "M", "F"],
    "Salary": [85, 90, 78, 85, 92, 88, 75, 80],
    "Age": [23, 25, 30, 22, 28, 26, 21, 27],
    "JoinDate": pd.to_datetime([
        "2020-01-10", "2020-02-15", "2021-03-20", "2021-04-10",
        "2020-05-30", "2020-06-25", "2021-07-15", "2021-08-01"
    ])
})  

```

```python
df.groupby("Department")["Salary"].mean()
```

This says:  
> “Group by Department, then calculate average Salary for each group.”

---

In [1]:
import pandas as pd

In [10]:
df = pd.DataFrame({
    "Department": ["HR", "HR", "IT", "IT", "Marketing", "Marketing", "Sales", "Sales"],
    "Team": ["A", "A", "B", "B", "C", "C", "D", "D"],
    "Gender": ["M", "F", "M", "F", "M", "F", "M", "F"],
    "Salary": [85, 90, 78, 85, 92, 88, 75, 80],
    "Age": [23, 25, 30, 22, 28, 26, 21, 27],
    "JoinDate": pd.to_datetime([
        "2020-01-10", "2020-02-15", "2021-03-20", "2021-04-10",
        "2020-05-30", "2020-06-25", "2021-07-15", "2021-08-01"
    ])
})  


In [11]:
df

,Department,Team,Gender,Salary,Age,JoinDate
0,HR,A,M,85,23,2020-01-10
1,HR,A,F,90,25,2020-02-15
2,IT,B,M,78,30,2021-03-20
3,IT,B,F,85,22,2021-04-10
4,Marketing,C,M,92,28,2020-05-30
5,Marketing,C,F,88,26,2020-06-25
6,Sales,D,M,75,21,2021-07-15
7,Sales,D,F,80,27,2021-08-01


## Common Aggregation Functions

```python
df.groupby("Team")["Salary"].mean()     # Average per team
df.groupby("Team")["Salary"].sum()      # Total score
df.groupby("Team")["Salary"].count()    # How many entries
df.groupby("Team")["Salary"].min()
df.groupby("Team")["Salary"].max()
```

To group by multiple columns:

```python
df.groupby(["Team", "Gender"])["Salary"].mean()
```

---

In [12]:
df.groupby("Department")["Salary"].mean()

Department
HR           87.5
IT           81.5
Marketing    90.0
Sales        77.5
Name: Salary, dtype: float64

In [13]:
type(df.groupby("Department")["Salary"].mean())

pandas.core.series.Series

In [14]:
df.groupby("Department")["Salary"].sum()

Department
HR           175
IT           163
Marketing    180
Sales        155
Name: Salary, dtype: int64

In [15]:
df.groupby("Department")["Salary"].count()

Department
HR           2
IT           2
Marketing    2
Sales        2
Name: Salary, dtype: int64

In [16]:
df.groupby("Department")["Salary"].min()

Department
HR           85
IT           78
Marketing    88
Sales        75
Name: Salary, dtype: int64

In [17]:
df.groupby("Department")["Salary"].max()

Department
HR           90
IT           85
Marketing    92
Sales        80
Name: Salary, dtype: int64

## Custom Aggregations with `.agg()`

Apply **multiple functions** at once like this:

```python
df.groupby("Team")["Salary"].agg(["mean", "max", "min"])
```
In pandas, .agg and .aggregate are exactly the same — they're aliases for the same method

Name your own functions:

```python
df.groupby("Team")["Salary"].agg(
    avg_score="mean",
    high_score="max"
)
```

Apply different functions to different columns:

```python
df.groupby("Team").agg({
    "Salary": "mean",
    "Age": "max"
})
```

---


In [18]:
df.groupby("Team")["Salary"].agg(["mean", "max", "min"])

,mean,max,min
Team,,,
A,87.5,90,85
B,81.5,85,78
C,90.0,92,88
D,77.5,80,75


In [19]:
type(df.groupby("Team")["Salary"].agg(["mean", "max", "min"]))

pandas.core.frame.DataFrame

In [20]:
df.groupby("Team")["Salary"].agg(avg_score="mean", high_score="max", low_score="min")

,avg_score,high_score,low_score
Team,,,
A,87.5,90,85
B,81.5,85,78
C,90.0,92,88
D,77.5,80,75


## Transform vs Aggregate vs Filter

| Operation | Returns | When to Use |
|-----------|---------|-------------|
| `.aggregate()` | Single value per group | Summary (like mean) |
| `.transform()` | Same shape as original | Add new column based on group |
| `.filter()`    | Subset of rows | Keep/discard whole groups |

### `.transform()` Example:

```python
df["Team Avg"] = df.groupby("Team")["Salary"].transform("mean")
```

Now each row gets its **team average** — great for comparisons!

### `.filter()` Example:

```python
df.groupby("Team").filter(lambda x: x["Salary"].mean() > 80)
```

Only keeps teams with average score > 80.

---

In [21]:
df["Team Avg"] = df.groupby("Team")["Salary"].transform("mean")

In [22]:
df

,Department,Team,Gender,Salary,Age,JoinDate,Team Avg
0,HR,A,M,85,23,2020-01-10,87.5
1,HR,A,F,90,25,2020-02-15,87.5
2,IT,B,M,78,30,2021-03-20,81.5
3,IT,B,F,85,22,2021-04-10,81.5
4,Marketing,C,M,92,28,2020-05-30,90.0
5,Marketing,C,F,88,26,2020-06-25,90.0
6,Sales,D,M,75,21,2021-07-15,77.5
7,Sales,D,F,80,27,2021-08-01,77.5


In [23]:
df.groupby("Team").filter(lambda x: x["Salary"].mean()>80)

,Department,Team,Gender,Salary,Age,JoinDate,Team Avg
0,HR,A,M,85,23,2020-01-10,87.5
1,HR,A,F,90,25,2020-02-15,87.5
2,IT,B,M,78,30,2021-03-20,81.5
3,IT,B,F,85,22,2021-04-10,81.5
4,Marketing,C,M,92,28,2020-05-30,90.0
5,Marketing,C,F,88,26,2020-06-25,90.0


## Summary

- `.groupby()` helps you summarize large datasets by category  
- Use `mean()`, `sum()`, `count()`, `.agg()` for custom metrics  
- `.transform()` adds values back to original rows  
- `.filter()` keeps only groups that meet conditions